# Assignment 2: Milestone I Natural Language Processing
## Task 2&3
#### Student Name: Ayush Kamleshbhai Patel
#### Student ID: 3891013

Date: XXXX

Version: 1.0

Environment: Python 3 and Jupyter notebook

Libraries used: please include all the libraries you used in your assignment, e.g.,:
* pandas
* re
* numpy

## Introduction
You should give a brief information of this assessment task here.

## Importing libraries 

In [1]:
# Code to import libraries as you need in this assessment, e.g.,
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from itertools import chain
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from gensim.models import FastText
from sklearn.decomposition import PCA
import gc
from sklearn.linear_model import LogisticRegression

# import umap.umap_ as umap



## Task 2. Generating Feature Representations for Job Advertisement Descriptions

#### Loading the data

In [2]:
with open('description.txt') as txtf:
    article_txts = txtf.read().splitlines() # reading a list of strings, each for a document/article
tk_description = [a.split(' ') for a in article_txts]

In [3]:
words = list(chain.from_iterable(tk_description))
# set of unique words
vocab = sorted(list(set(words)))
# total number of the vocabulary
len(vocab)


5168

In [4]:
jobs_data = pd.read_csv('jobs_data.csv')
webindex = jobs_data['Webindex']
# Define the mapping
category_to_int_mapping = {
    'Accounting_Finance': 0,
    'Engineering': 1,
    'Healthcare_Nursing': 2,
    'Sales': 3
}

# Replace the labels with their integer values
jobs_data['category_encoded'] = jobs_data['Category'].map(category_to_int_mapping)
jobs_data['category_encoded'] = jobs_data['category_encoded'].astype(int)
jobs_data.head(5)

,Webindex,Title,Company,Description,Tokenized Description,Tokenized Title,Tokenized Company,Category,category_encoded
0,68997528,Finance / Accounts Asst Bromley to ****k,First Recruitment Services,Accountant (partqualified) to **** p.a. South ...,"['accountant', 'partqualified', 'south', 'east...","['finance', 'accounts', 'asst', 'bromley', 'to...","['first', 'recruitment', 'services']",Accounting_Finance,0
1,68063513,Fund Accountant Hedge Fund,Austin Andrew Ltd,One of the leading Hedge Funds in London is cu...,"['hedge', 'funds', 'london', 'recruiting', 'fu...","['fund', 'accountant', 'hedge', 'fund']","['austin', 'andrew', 'ltd']",Accounting_Finance,0
2,68700336,Deputy Home Manager,Caritas,An exciting opportunity has arisen to join an ...,"['exciting', 'arisen', 'establish', 'provider'...","['deputy', 'home', 'manager']",['caritas'],Healthcare_Nursing,2
3,67996688,Brokers Wanted Imediate Start,OneTwoTrade,OneTwoTrade is expanding their Sales Team and ...,"['expanding', 'recruiting', 'junior', 'trainee...","['brokers', 'wanted', 'imediate', 'start']",['onetwotrade'],Accounting_Finance,0
4,71803987,RGN Nurses (Hospitals) Penarth,Swiis Healthcare,RGN Nurses (Hospitals) Immediate fulltime and ...,"['rgn', 'nurses', 'hospitals', 'fulltime', 'pa...","['rgn', 'nurses', 'hospitals', 'penarth']","['swiis', 'healthcare']",Healthcare_Nursing,2


### Bag Of Words Model

#### Creating Binary Vectors

In [5]:
# Code to perform the task...
joined_description = [' '.join(review) for review in tk_description]
bVectorizer = CountVectorizer(analyzer = "word",binary = True,vocabulary = vocab) # initialise the CountVectorizer
binary_features = bVectorizer.fit_transform(joined_description)
binary_features.shape

(776, 5168)

#### Creating Count Vectors

In [6]:
cVectorizer = CountVectorizer(analyzer = "word",vocabulary = vocab) # initialised the CountVectorizer
count_features = cVectorizer.fit_transform(joined_description)
count_features.shape

(776, 5168)

In [7]:
count_features_df = pd.DataFrame(count_features.toarray(), columns=cVectorizer.get_feature_names_out())
# print out samples
count_features_df.head(3)

,aap,aaron,aat,abb,abenefit,aberdeen,abi,abilities,abreast,abroad,...,years,yeovil,yn,york,yorkshire,youmust,young,younger,yrs,zest
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Saving it into the file

In [8]:
count_features = cVectorizer.fit_transform(joined_description).toarray()

def save_count_vector(count_features, webindex, filename):
    with open(filename, 'w') as f:
        for i in range(len(count_features)):
            f.write('#' + str(webindex[i]) + ',')
            for j in range(len(count_features[i])):
                if count_features[i][j] != 0:
                    f.write(str(j) + ':' + str(count_features[i][j]) + ',')
            f.write('\n')
    f.close()
    print('Count vector representation saved to ' + filename)


save_count_vector(count_features, webindex, 'count_vectors.txt')

Count vector representation saved to count_vectors.txt


#### Creating TF-IDF Vectors

In [9]:
tVectorizer = TfidfVectorizer(analyzer = "word",vocabulary = vocab) # initialised the TfidfVectorizer
tfidf_features = tVectorizer.fit_transform(joined_description) # generate the tfidf vector representation for all articles
tfidf_features.shape

(776, 5168)

In [10]:
def write_vectorFile(data_features,filename):
    num = data_features.shape[0] # the number of document
    out_file = open(filename, 'w') # creates a txt file and open to save the vector representation
    for a_ind in range(0, num): # loop through each article by index
        for f_ind in data_features[a_ind].nonzero()[1]: # for each word index that has non-zero entry in the data_feature
            value = data_features[a_ind][0,f_ind] # retrieve the value of the entry from data_features
            out_file.write("{}:{} ".format(f_ind,value)) # write the entry to the file in the format of word_index:value
        out_file.write('\n') # start a new line after each article
    out_file.close() # close the file   

In [11]:
tfidf_features_file = "jobs_data_vector.txt" # file name of the tfidf vector

write_vectorFile(tfidf_features,tfidf_features_file) # write the tfidf vector to file

## Creating Model based on word embeddings

#### Creating and Saving FastText Model

In [12]:
# 1. Set the corpus file names/path
corpus_file = 'description.txt'

# 2. Initialise the Fast Text model
descFT = FastText(vector_size=50) 

# 3. build the vocabulary
descFT.build_vocab(corpus_file=corpus_file)

# 4. train the model
descFT.train(
    corpus_file=corpus_file, 
    epochs=descFT.epochs,
    total_examples=descFT.corpus_count, 
    total_words=descFT.corpus_total_words,
)

print(descFT)
# embedding_size = 200
# window_size = 5
# min_word = 5
# down_sampling = 1e-2
 
# %time
# descFT = FastText(tk_description,
#                       vector_size=embedding_size,
#                       window=window_size,
#                       min_count=min_word,
#                       sample=down_sampling,
#                       workers = 5,
#                       sg=1,
#                       epochs=100)

FastText<vocab=2741, vector_size=50, alpha=0.025>


In [13]:
descFT_wv = descFT.wv
print(descFT_wv)

FastTextKeyedVectors<vector_size=50, 2741 keys>


In [14]:
#Saving the model
from gensim.models import Word2Vec
descFT.save("models/FastText/descFT.model")
# print(f'Successfully saved {FastTextModel}')


In [15]:
descFT = FastText.load("models/FastText/descFT.model")
print(descFT)
descFT_wv= descFT.wv

FastText<vocab=2741, vector_size=50, alpha=0.025>


In [16]:
def docvecs(embeddings, docs):
    vecs = np.zeros((len(docs), embeddings.vector_size))
    for i, doc in enumerate(docs):
        valid_keys = [term for term in doc if term in embeddings.key_to_index]
        if valid_keys:  # Check if the list is not empty
            docvec = np.vstack([embeddings[term] for term in valid_keys])
            docvec = np.sum(docvec, axis=0)
            vecs[i,:] = docvec
    return vecs


In [17]:
descFT_dvs = docvecs(descFT_wv, jobs_data['Tokenized Description'])

### Building the weighted (i.e., TF-IDF weighted) and unweighted vector representation

* the following `gen_docVecs` function takes the word embeddings dictionary, the tokenized text of articles, and the tfidf weights (list of word:weight dictionaries, one for each article) as arguments, and generates the document embeddings:
 1. creates an empty dataframe `docs_vectors` to store the document embeddings of articles
  2. it loop through every tokenized text:
    - creates an empty dataframe `temp` to store all the word embeddings of the article
    - for each word that exists in the word embeddings dictionary/keyedvectors, 
        - if the argument `tfidf` weights are empty `[]`, it sets the weight of the word as 1
        - otherwise, retrieve the weight of the word from the corresponding word:weight dictionary of the article from  `tfidf`
    - row bind the weighted word embedding to `temp`
    - takes the sum of each column to create the document vector, i.e., the embedding of an article
    - append the created document vector to the list of document vectors

In [18]:
import pandas as pd
import numpy as np

def gen_docVecs(wv,tk_txts):
    docs_vectors = pd.DataFrame() # creating empty final dataframe
    #stopwords = nltk.corpus.stopwords.words('english') # if we haven't pre-processed the articles, it's a good idea to remove stop words

    for i in range(0,len(tk_txts)):
        tokens = tk_txts[i]
        temp = pd.DataFrame()  # creating a temporary dataframe(store value for 1st doc & for 2nd doc remove the details of 1st & proced through 2nd and so on..)
        for w_ind in range(0, len(tokens)): # looping through each word of a single document and spliting through space
            try:
                word = tokens[w_ind]
                word_vec = wv[word] # if word is present in embeddings(goole provides weights associate with words(300)) then proceed
                temp = pd.concat([temp, pd.Series(word_vec)], ignore_index = True)
            except:
                pass
        doc_vector = temp.sum() # take the sum of each column
        docs_vectors = pd.concat([docs_vectors, doc_vector], ignore_index = True)
    return docs_vectors


#### Unweighted Document Vectors

In [19]:
# generate document embeddings
# descFT_vw = descFT.wv
# descFT_dvs = gen_docVecs(descFT_vw, jobs_data['Tokenized Description'])
# descFT_dvs.isna().any().sum()

# bbcFT = FastText.load("bbcFT.model")
# print(bbcFT)
# bbcFT_wv= bbcFT.wv

In [20]:
# explore feature space
# features = descFT_dvs.to_numpy()
# bbcFT_dvs = docvecs(bbcFT_wv, df['tk_text'])


In [21]:
def plotTSNE(labels,features): # features as a numpy array, each element of the array is the document embedding of an article
    categories = sorted(labels.unique())
    # Sampling a subset of our dataset because t-SNE is computationally expensive
    SAMPLE_SIZE = int(len(features) * 0.1)
    np.random.seed(0)
    indices = np.random.choice(range(len(features)), size=SAMPLE_SIZE, replace=False)
    projected_features = TSNE(n_components=2, random_state=0).fit_transform(features[indices])
    colors = ['pink', 'green', 'midnightblue', 'orange', 'darkgrey']
    for i in range(0,len(categories)):
        points = projected_features[(labels[indices] == categories[i])]
        plt.scatter(points[:, 0], points[:, 1], s=30, c=colors[i], label=categories[i])
    plt.title("Feature vector for each article, projected on 2 dimensions.",
              fontdict=dict(fontsize=15))
    plt.legend()
    plt.show()

In [22]:
# features = descFT_dvs.to_numpy()
# plotTSNE(jobs_data['Category'],features)

# features = bbcFT_dvs
features = descFT_dvs
plotTSNE(jobs_data['Category'],features)

: 

In [ ]:
features

array([[ 47.354122  ],
       [ 69.44636   ],
       [ 78.27909   ],
       [ 56.67145   ],
       [103.4588    ],
       [139.86433   ],
       [ 98.91203   ],
       [148.6745    ],
       [ 74.91358   ],
       [133.72914   ],
       [183.32669   ],
       [ 18.865896  ],
       [195.4465    ],
       [135.99606   ],
       [ 86.050285  ],
       [ 54.010113  ],
       [229.00507   ],
       [371.71237   ],
       [197.3951    ],
       [ 42.22616   ],
       [378.58243   ],
       [243.41043   ],
       [179.77446   ],
       [176.58612   ],
       [197.07759   ],
       [187.30481   ],
       [183.46584   ],
       [ 62.91546   ],
       [ 94.4679    ],
       [ 81.08836   ],
       [135.24193   ],
       [ 93.887115  ],
       [ 75.359116  ],
       [ 86.145226  ],
       [  5.4172144 ],
       [166.19101   ],
       [197.79918   ],
       [120.86401   ],
       [236.93898   ],
       [369.20142   ],
       [261.20776   ],
       [142.26173   ],
       [187.99835   ],
       [ 37

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# import umap.umap_ as umap
# from sklearn.preprocessing import LabelEncoder

# # Encode the string labels to integers
# encoder = LabelEncoder()
# labels_encoded = encoder.fit_transform(jobs_data['Category'])

# # UMAP dimensionality reduction
# reducer = umap.UMAP()
# features_umap = reducer.fit_transform(features)

# # Plotting the results
# plt.figure(figsize=(10, 8))
# scatter = plt.scatter(features_umap[:, 0], features_umap[:, 1], c=labels_encoded, cmap='Spectral', s=5)
# plt.colorbar(scatter)
# plt.title('UMAP Projection of Features')
# plt.show()


In [ ]:
# try:
#     features = descFT_dvs.to_numpy() # convert the document vector dataframe to a numpy array
# except:
#     # used `docvec` so already is a numpy array
#     features = descFT_dvs
# plotTSNE(jobs_data['Category'],features) # plot the tSNE to have a look

In [ ]:
seed = 3891013
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test,train_indices,test_indices = train_test_split(descFT_dvs, jobs_data['Category'], list(range(0,len(jobs_data))),test_size=0.3, random_state=seed)

model = LogisticRegression(max_iter = 1000,random_state=seed)
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.30472103004291845

#### Unweighted Document Vectors

In [ ]:
def read_vocab(vocab_file):
    vocab = {}
    with open(vocab_file) as f:
        for line in f:
            (word, index) = line.split(':')
            vocab[word.strip()] = int(index)
    return {v: k for k, v in vocab.items()}

# Generates the w_index:word dictionary
voc_fname = 'vocab.txt'
voc_dict = read_vocab(voc_fname)
voc_dict

{0: 'aap',
 1: 'aaron',
 2: 'aat',
 3: 'abb',
 4: 'abenefit',
 5: 'aberdeen',
 6: 'abi',
 7: 'abilities',
 8: 'abreast',
 9: 'abroad',
 10: 'absence',
 11: 'absolute',
 12: 'ac',
 13: 'aca',
 14: 'academic',
 15: 'academy',
 16: 'acca',
 17: 'accept',
 18: 'acceptable',
 19: 'acceptance',
 20: 'accepted',
 21: 'access',
 22: 'accessible',
 23: 'accident',
 24: 'accommodates',
 25: 'accommodation',
 26: 'accomplished',
 27: 'accordance',
 28: 'account',
 29: 'accountabilities',
 30: 'accountability',
 31: 'accountable',
 32: 'accountancy',
 33: 'accountant',
 34: 'accountants',
 35: 'accounting',
 36: 'accounts',
 37: 'accreditation',
 38: 'accredited',
 39: 'accruals',
 40: 'accuracy',
 41: 'accurate',
 42: 'accurately',
 43: 'achievable',
 44: 'achieve',
 45: 'achieved',
 46: 'achievement',
 47: 'achievements',
 48: 'achiever',
 49: 'achieving',
 50: 'acii',
 51: 'acquired',
 52: 'acquisition',
 53: 'acquisitions',
 54: 'act',
 55: 'acting',
 56: 'action',
 57: 'actions',
 58: 'action

the `doc_wordweights` function takes the tfidf document vector file, as well as the w_index:word dictionary, creates the mapping between w_index and the actual word, and creates a dictionary of word:weight or each unique word appear in the document.

In [ ]:
def doc_wordweights(fName_tVectors, voc_dict):
    tfidf_weights = [] # a list to store the  word:weight dictionaries of documents
    
    with open(fName_tVectors) as tVecf: 
        tVectors = tVecf.read().splitlines() # each line is a tfidf vector representation of a document in string format 'word_index:weight word_index:weight .......'
    for tv in tVectors: # for each tfidf document vector
        tv = tv.strip()
        weights = tv.split(' ') # list of 'word_index:weight' entries
        weights = [w.split(':') for w in weights] # change the format of weight to a list of '[word_index,weight]' entries
        wordweight_dict = {voc_dict[int(w[0])]:w[1] for w in weights} # construct the weight dictionary, where each entry is 'word:weight'
        tfidf_weights.append(wordweight_dict) 
    return tfidf_weights

fName_tVectors = 'jobs_data_vector.txt'
tfidf_weights = doc_wordweights(fName_tVectors, voc_dict)
tfidf_weights

[{'terms': '0.1308521762366382',
  'south': '0.20612279476719383',
  'requirement': '0.1249811292375601',
  'reconciliation': '0.14183179957804745',
  'qualified': '0.07821166567368464',
  'purchase': '0.13660500449583682',
  'previous': '0.08245826258151072',
  'possess': '0.10970668693524623',
  'phone': '0.11922830097836144',
  'person': '0.09046563231320097',
  'permanent': '0.08174449947711492',
  'payment': '0.13824362212075778',
  'partqualified': '0.173326820839609',
  'part': '0.07189780182251101',
  'offices': '0.11518385918795168',
  'negotiating': '0.15985275501239027',
  'modern': '0.13218581222143622',
  'manufacturing': '0.09726367012728679',
  'london': '0.16839249893971672',
  'letter': '0.12835773375082868',
  'ledger': '0.27321000899167364',
  'ideal': '0.08802432794046589',
  'handling': '0.24392130714952515',
  'fully': '0.1061919231001392',
  'exceptional': '0.12835773375082868',
  'email': '0.07884175705492424',
  'east': '0.20514266820633364',
  'duties': '0.082

Since we are going to explore the use of different language model for document classification in this activity, we will need to generate docment embeddings based on word embeddings a couple of times. 
Let's construct a function for this so that we can resue it later.

The following function takes:
* wv, an word:embedding dictionary or KeyedVectors; and 
* tk_txts, a list of tokenized texts, each of an article

as argument, it then does the following to generate the list of embedding vector representations, each for an article:
1. creates an empty dataframe `docs_vectors` to store the document embeddings of articles
2. it loop through every tokenized text:
    - creates an empty dataframe `temp` to store all the word embeddings of the article
    - for each word that exists in the word embeddings dictionary/keyedvectors, row bind the word embedding to `temp`
    - take the sum of each column to create the document vector, i.e., the embedding of an article
    - append the created document vector to the list of document vectors

In [ ]:
def gen_docVecs(wv,tk_txts,tfijob_ad = []): # generate vector representation for documents
    docs_vectors = pd.DataFrame() # creating empty final dataframe
    #stopwords = nltk.corpus.stopwords.words('english') # removing stop words

    for i in range(0,len(tk_txts)):
        tokens = list(set(tk_txts[i])) # get the list of distinct words of the document

        temp = pd.DataFrame()  # creating a temporary dataframe(store value for 1st doc & for 2nd doc remove the details of 1st & proced through 2nd and so on..)
        for w_ind in range(0, len(tokens)): # looping through each word of a single document and spliting through space
            try:
                word = tokens[w_ind]
                word_vec = wv[word] # if word is present in embeddings(goole provides weights associate with words(300)) then proceed
                
                if tfijob_ad != []:
                    word_weight = float(tfijob_ad[i][word])
                else:
                    word_weight = 1
                temp = temp.append(pd.Series(word_vec*word_weight), ignore_index = True) # if word is present then append it to temporary dataframe
            except:
                pass
        doc_vector = temp.sum() # take the sum of each column(w0, w1, w2,........w300)
        docs_vectors = docs_vectors.append(doc_vector, ignore_index = True) # append each document value to the final dataframe
    return docs_vectors

In [ ]:
weighted_descFT_company = gen_docVecs(descFT_vw,jobs_data['Tokenized Company'],tfidf_weights)
weighted_descFT_title = gen_docVecs(descFT_vw,jobs_data['Tokenized Title'],tfidf_weights)
weighted_descFT_description = gen_docVecs(descFT_vw,jobs_data['Tokenized Description'],tfidf_weights)
weighted_descFT_description

NameError: name 'tfidf_weights' is not defined

In [ ]:
def plotTSNE(labels,features): # features as a numpy array, each element of the array is the document embedding of an article
    categories = sorted(labels.unique())
    # Sampling a subset of our dataset because t-SNE is computationally expensive
    SAMPLE_SIZE = int(len(features) * 0.3)
    np.random.seed(0)
    indices = np.random.choice(range(len(features)), size=SAMPLE_SIZE, replace=False)
    projected_features = TSNE(n_components=2, random_state=0).fit_transform(features[indices].astype(int))
    colors = ['pink', 'green', 'midnightblue', 'orange', 'darkgrey']
    for i in range(0,len(categories)):
        points = projected_features[(labels[indices] == categories[i])]
        plt.scatter(points[:, 0], points[:, 1], s=30, c=colors[i], label=categories[i])
    plt.title("Feature vector for each article, projected on 2 dimensions.",
              fontdict=dict(fontsize=15))
    plt.legend()
    plt.show()

In [ ]:
seed = 3891013
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

models = weighted_descFT_description
# model_names = "Weighted Pretrained FastText with Description"

dv = models
# name = model_names
# features = dv.to_numpy() # convert the dataframe stored features to an numpy array
# print(len(features))
# print(name + ": tSNE 2 dimensional projected Feature space")
# # plotTSNE(jobs_data['Category'],features)
    
# # creating training and test split
# X_train, X_test, y_train, y_test,train_indices,test_indices = train_test_split(dv, jobs_data['Category'], list(range(0,len(jobs_data))),test_size=0.33, random_state=seed)

# model = LogisticRegression(max_iter = 2000,random_state=seed)
# # X_train = np.array(X_train).reshape(-1, 1)
# # y_train = np.array(y_train).reshape(-1, 1)
# # y_test = np.array(y_test).reshape(-1, 1)
# # X_test = np.array(X_test).reshape(-1, 1)
# model.fit(X_train, y_train)
# print("Accuracy: ", model.score(X_test, y_test))
# print('\n\n')
print('len dv:', len(dv))


0
Weighted Pretrained FastText with Description: tSNE 2 dimensional projected Feature space


ValueError: Found input variables with inconsistent numbers of samples: [0, 776, 776]

### Saving outputs
Save the count vector representation as per spectification.
- count_vectors.txt

In [ ]:
# code to save output data...

## Task 3. Job Advertisement Classification

...... Sections and code blocks on buidling classification models based on different document feature represetations. 
Detailed comparsions and evaluations on different models to answer each question as per specification. 

<span style="color: red"> You might have complex notebook structure in this section, please feel free to create your own notebook structure. </span>

In [ ]:
# Code to perform the task...


## Summary
Give a short summary and anything you would like to talk about the assessment tasks here.

## Couple of notes for all code blocks in this notebook
- please provide proper comment on your code
- Please re-start and run all cells to make sure codes are runable and include your output in the submission.   
<span style="color: red"> This markdown block can be removed once the task is completed. </span>